In [ ]:
from nbdev import *
# default_exp sources

%load_ext autoreload
%autoreload 2

from utilities.ipynb_docgen import *
from nbdev.showdoc import show_doc

# Sources and Weights
> Define PointSource, load weight tables to combine with photon data

We use the full-sky catalog analysis model to evaluate the predicted flux from a source of interest with respect to the
background, the combined fluxes from all other sources. We choose the following binning:

* energy:  4/decade from 100 MeV to 1 TeV 
* event type: Front and Back  
* Angular position: HEALPix, currently nside=64, for 1 degree-square pixels,

### Pointlike generation 
A procedure, currently only for pointlike, [see source_weighsts](https://github.com/tburnett/pointlike/blob/master/python/uw/like2/source_weights.py) packs this table with the source name and position, into a pickled dict.

#### Original format -- fixed 1-degree pixel
```
        outdict = dict(
            model_name = '/'.join(os.getcwd().split('/')[-2:]),
            radius=self.radius,
            nside=self.nside,
            order='NEST',
            energy_bins=self.energy_bins,
            source_name= self.source.name,
            source_lb=galactic(self.source.skydir),
            roi_lb  = galactic(self.roi.roi_dir),
            roi_name=self.roi.name,
            pixels= pixels,
            weights = weights,
        )

```
#### New format -- varying pixel size depending on PSF width


This is unpacked by the class `WeightMan`

This table is used with the data, as a simple lookup: A weight is assigned to each photon according to which energy, event type or HEALPix pixel it lands in.

### Accounting for variations from neighboring sources

Consider the case where sources $S_1$ and $S_2$ have overlapping pixels. For a given pixel the corresponding weights are
$w_1$ and $w_2$, and we investigate the effect on $S_1$ from a fractional variation $\alpha_2 \ne 0$ of $S_2$, such that
its flux for that pixel, $s_2$, becomes $(1+\alpha )\ s_2$. With the background $b$, the flux of all
sources besides $S_1$ and $S_2$, we have for the $S_1$ weight,
$$ w_1 = \frac{s_1}{s_1+s_2+b}\ \ ,$$ and similarly for $S_2$.

Replacing $s_2$ with $(1+\alpha ) s_2$, we have for the modified weight $w_1'$ that we should use for  $S_1$,
$$w'_1 = \frac{w_1}{1+\alpha_2\ w_2}\ \ .   $$


In [ ]:
# export
import os, sys,  pickle, healpy
from pathlib import Path
import numpy as np
from scipy.integrate import quad
from astropy.coordinates import SkyCoord, Angle
from wtlike.config import *

In [ ]:
# export
def check_weights(config, source):
    """
    Check that weights for the source are available: if so, return the weight file name

    - source -- A PointSource object with information on source location

    Returns the filepath to the file if successful, otherwise, print a message abount available files
    """
    weight_files = config.wtlike_data/'weight_files'
    assert weight_files.is_dir(), f'Expect {weight_files} to be a directory'
    weight_file = weight_files/ (source.filename+'_weights.pkl')
    if not weight_file.exists():
        available = np.array(list(map(lambda p: p.name[:p.name.find('_weights')],
                          weight_files.glob('*_weights.pkl'))))
        print(f'{source} not found in list of weight files at\n\t {weight_files}.\n Available:\n{available}',
             file = sys.stderr)
        return None
    return weight_file

In [ ]:
# export
# def load_weights(config, filename, ):
#     """Load the weight informaton

#     filename: pickled dict with map info

#     """
#     # load a pickle containing weights, generated by pointlike
#     assert os.path.exists(filename),f'File {filename} not found.'
#     with open(filename, 'rb') as file:
#         wtd = pickle.load(file, encoding='latin1')
#     assert type(wtd)==dict, 'Expect a dictionary'
#     test_elements = 'energy_bins pixels weights nside model_name radius order roi_name'.split()
#     assert np.all([x in wtd.keys() for x in test_elements]),f'Dict missing one of the keys {test_elements}'
#     if config.verbose>0:
#         print(f'Load weights from file {os.path.realpath(filename)}')
#         pos = wtd['source_lb']
#         print(f'\tFound: {wtd["source_name"]} at ({pos[0]:.2f}, {pos[1]:.2f})')
#     # extract pixel ids and nside used
#     wt_pix   = wtd['pixels']
#     nside_wt = wtd['nside']

#     # merge the weights into a table, with default nans
#     # indexing is band id rows by weight pixel columns
#     # append one empty column for photons not in a weight pixel
#     # calculated weights are in a dict with band id keys
#     wts = np.full((32, len(wt_pix)+1), np.nan, dtype=np.float32)
#     weight_dict = wtd['weights']
#     for k in weight_dict.keys():
#         t = weight_dict[k]
#         if len(t.shape)==2:
#             t = t.T[0] #???
#         wts[k,:-1] = t
#     return wts , wt_pix , nside_wt

In [ ]:
# export
def _add_weights(config, wts, wt_pix, nside_wt, photon_data):
    """ get the photon pixel ids, convert to NEST (if not already) and right shift them
        add 'weight', remove 'band', 'pixel'
    """
    if not config.nest:
        # data are RING
        photon_pix = healpy.ring2nest(config.nside, photon_data.nest_index.values)
    else:
        photon_pix = photon_data.nest_index.values
    to_shift = 2*int(np.log2(config.nside/nside_wt));
    shifted_pix =   np.right_shift(photon_pix, to_shift)
    bad = np.logical_not(np.isin(shifted_pix, wt_pix))
    if config.verbose>0 & sum(bad)>0:
        print(f'\tApplying weights: {sum(bad)} / {len(bad)} photon pixels are outside weight region')
    if sum(bad)==len(bad):
        a = np.array(healpy.pix2ang(nside_wt, wt_pix, nest=True, lonlat=True)).mean(axis=1).round(1)
        b = np.array(healpy.pix2ang(nside_wt, shifted_pix, nest=True, lonlat=True)).mean(axis=1).round(1)

        raise Exception(f'There was no overlap of the photon data at {b} and the weights at {a}')
    shifted_pix[bad] = 12*nside_wt**2 # set index to be beyond pixel indices

    # find indices with search and add a "weights" column
    # (expect that wt_pix are NEST ordering and sorted)
    weight_index = np.searchsorted(wt_pix,shifted_pix)
    band_index = np.fmin(31, photon_data.band.values) #all above 1 TeV into last bin

    # final grand lookup -- isn't numpy wonderful!
    photon_data.loc[:,'weight'] = self.wts[tuple([band_index, weight_index])]

    # don't need these columns now (add flag to config to control??)
#     photon_data.drop(['band', 'pixel'], axis=1)

    if config.verbose>1:
        print(f'\t{sum(np.isnan(photon_data.weight.values))} events without weight')

In [ ]:
# export
def add_weights(config,  photon_data, source): # nbins=50):
    """ add weights for the source to the photon data

    - photon_data -- DataFrame with photon data

    - source -- `PointSource` object

    """
    assert source.wtman is not None, 'Source weights not loaded'
#     weight_file =  check_weights(config,  source)
#     if source.wtman is None:
#         raise Exception(f'No weight info in {source}')

#     ## NEW
#     wtman = WeightMan(config, filename=weight_file)
    photon_data = source.wtman.add_weights(photon_data)

    ## OLD
#     wts, wt_pix, nside_wt = load_weights(config, weight_file)
#     _add_weights(config, wts, wt_pix, nside_wt, photon_data)

    #return np.histogram(photon_data.weight.values, np.linspace(0,1,nbins+1))[0]

In [ ]:
#export
class WeightMan(dict):
    """ Weight Management

    * Load weight tables
    * Assign weights to photons
    """

    def __init__(self, config,  source=None, filename=None,):
        """
        TODO: find filename given source
        """
        wtpath =Path(config.wtlike_data)/'weight_files'
        assert wtpath.is_dir(), f' {wtpath} not an existing file path'
        # load a pickle containing weights, generated by pointlike

        if isinstance(source, PointSource) or source.__class__.__name__=='PointSource':
            wtpath =Path(config.wtlike_data)/'weight_files'
            assert wtpath.is_dir(), f' {wtpath} not an existing file path'
            full_filename = wtpath/(source.filename+'_weights.pkl')
            self.source=source

        elif filename is not None:
            full_filename = wtpath/filename
            self.source = None

        else:
            raise Exception('WeightMan: expected source or filename')

        assert (full_filename).is_file(),f'File {filename} not found at {wtpath}'

        with open(full_filename, 'rb') as file:
            wtd = pickle.load(file, encoding='latin1')
        assert type(wtd)==dict, 'Expect a dictionary'
        self.update(wtd)
        self.__dict__.update(wtd)
        self.filename=filename
        self.config = config
#         pos = self['source_lb']
#         print(f'\tSource is {self["source_name"]} at ({pos[0]:.2f}, {pos[1]:.2f})')

        # check format--old has pixels, weights at tome
        srcfile = f'file "{self.filename}"' if self.source is None else f'file from source "{source.filename}"_weights.pkl'

        if hasattr(self, 'nside'):
            self.format=0
            if config.verbose>0:
                print(f'WeightMan: {srcfile} old format, nside={self.nside}')

            test_elements = 'energy_bins pixels weights nside model_name radius order roi_name'.split()
            assert np.all([x in wtd.keys() for x in test_elements]),f'Dict missing one of the keys {test_elements}'
            if config.verbose>0:
                print(f'Load weights from file {os.path.realpath(filename)}')
                pos = self['source_lb']
                print(f'\tFound: {self["source_name"]} at ({pos[0]:.2f}, {pos[1]:.2f})')
            # extract pixel ids and nside used
            self.wt_pix   = self['pixels']
            self.nside_wt = self['nside']

            # merge the weights into a table, with default nans
            # indexing is band id rows by weight pixel columns
            # append one empty column for photons not in a weight pixel
            # calculated weights are in a dict with band id keys
            self.wts = np.full((32, len(self.wt_pix)+1), np.nan, dtype=np.float32)
            weight_dict = self['weights']
            for k in weight_dict.keys():
                t = weight_dict[k]
                if len(t.shape)==2:
                    t = t.T[0] #???
                self.wts[k,:-1] = t

        else:
            self.format=1
            wtdict = self.wt_dict
            nsides = [v['nside'] for v in wtdict.values() ];

            if config.verbose>1:
                print(f'WeightMan: {srcfile} : new format, {len(nsides)} bamds'\
                      f' with nsides {nsides[0]} to {nsides[-1]}')
            if self.source is not None:
                self.source.fit_info = self.fitinfo
                if config.verbose>1:
                    print(f'\tAdded fit info {self.fitinfo} to source')

    def _new_format(self, photons):

        wt_tables =self.wt_dict
        data_nside=1024



        #photons = photons.rename(columns=dict(weight='old_wt'))
        photons.loc[:,'weight'] = np.nan

        if self.config.verbose>1:
            print(f'WeightMan: processing {len(photons):,} photons')

        def load_data( band_id):
            """ fetch pixels and weights for the band;
                adjust pixels to the band nside
                generate mask for pixels, weights
            """
            band = photons[photons.band==band_id] #.query('band== @band_id')
            wt_table = wt_tables[band_id]
            nside =  wt_table['nside']
            new_weights = wt_table['wts'].astype(np.float16)
            to_shift = int(2*np.log2(data_nside//nside))
            data_pixels = np.right_shift(band.nest_index, to_shift)
            wt_pixels=wt_table['pixels']
            good = np.isin( data_pixels, wt_pixels)
            if self.config.verbose>2:
                print(f'\t {band_id:2}: {len(band):8,} -> {sum(good ):8,}')
            return data_pixels, new_weights, good

        def set_weights(band_id):
            if band_id not in wt_tables.keys(): return

            data_pixels, new_weights, good = load_data(band_id)
            wt_pixels = wt_tables[band_id]['pixels']
            indicies = np.searchsorted( wt_pixels, data_pixels[good])
            new_wts = new_weights[indicies]
            # get subset of photons in this band, with new weights
            these_photons = photons[photons.band==band_id][good]
            these_photons.loc[:,'weight']=new_wts
            photons.loc[photons.band==band_id,'weight'] = (these_photons.weight).astype(np.float16)
    #         if self.config.verbose>1:
    #             print(f' -> {len(new_wts):8,}')

        for band_id in range(16):
            set_weights(band_id)

        return photons

    def add_weights(self, photons):
        """
        get the photon pixel ids, convert to NEST (if not already) and right shift them
        add 'weight', remove 'band', 'nest_index'

        """
#         if self.format==0:
#             self._old_format(photons)
#         else:
        photons = self._new_format(photons)

        # don't need these columns now (add flag to config to control??)
        photons.drop(['nest_index'], axis=1, inplace=True)

        if self.config.verbose>1:
            print(f'\t{sum(np.isnan(photons.weight.values)):,} events without weight')
        return photons

def weight_radius_plots(photons):
    """
    """
    import matplotlib.pyplot as plt

    fig, axx = plt.subplots(2,8, figsize=(16,5), sharex=True, sharey=True)
    plt.subplots_adjust(hspace=0.02, wspace=0)
    for id,ax in enumerate(axx.flatten()):
        subset = photons.query('band==@id & weight>0')
        ax.semilogy(subset.radius, subset.weight, '.', label=f'{id}');
        ax.legend(loc='upper right', fontsize=10)
        ax.grid(alpha=0.5)
    ax.set(ylim=(8e-4, 1.2), xlim=(0,4.9))
    plt.suptitle('Weights vs. radius per band')

In [ ]:
show_doc(WeightMan)

<h2 id="WeightMan" class="doc_header"><code>class</code> <code>WeightMan</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>WeightMan</code>(**`config`**, **`source`**=*`None`*, **`filename`**=*`None`*) :: `dict`

Weight Management

* Load weight tables
* Assign weights to photons

In [ ]:
#export      

class PointSource():
    """Manage the position and name of a point source
    """
    def __init__(self, name,  position=None, nickname=None, config=None,):
        """
           * position: [None] (l,b) tuple or None. if None, expect to be found by lookup
           * nickname [None] -- use for file name, especially weights (in config.weight_folder)
               if not set, convert name to file friendly format

        """
        self.name=name
        if position is None:
            if name.startswith('J'):
                # parse the name for (ra,dec)
                ra=(name[1:3]+'h'+name[3:7]+'m')
                dec = (name[7:10]+'d'+name[10:12]+'m')
                (ra,dec) = map(lambda a: float(Angle(a, unit=u.deg).to_string(decimal=True)),(ra,dec))
                skycoord = SkyCoord(ra, dec, unit='deg', frame='fk5')
            else:
                try:
                    skycoord = SkyCoord.from_name(name)
                except Exception as e:
                    print(f'PointSource: a source "{name}" was not recognized by SkyCoord: {e}', 
                          file=sys.stderr)
                    raise
            gal = skycoord.galactic
            self.l,self.b = (gal.l.value, gal.b.value)
        else:
            self.l,self.b = position
            skycoord = SkyCoord(self.l,self.b, unit='deg', frame='galactic')
        self.skycoord = skycoord
        self.nickname = nickname or self.filename
        
        self.config = config or Config()
        try:
            self.wtman = WeightMan(self.config, self)
            # add wtman attribute references
            self.__dict__.update(self.wtman.__dict__)
        except Exception as e:
            print(f'Unexpected Weigthman failure: {e}', file=sys.stderr)
            raise



            
    def __str__(self):
        return f'Source "{self.name}" at: (l,b)=({self.l:.3f},{self.b:.3f})'
    def __repr__(self): return str(self)

    @property
    def ra(self):
        sk = self.skycoord.transform_to('fk5')
        return sk.ra.value
    @property
    def dec(self):
        sk = self.skycoord.transform_to('fk5')
        return sk.dec.value

    @property
    def filename(self):
        """Modified name for file system"""
        return self.name.replace(' ', '_').replace('+','p') if getattr(self,'nickname',None) is None else self.nickname

    @classmethod
    def fk5(cls, name, position):
        """position: (ra,dec) tuple """
        ra,dec = position
        sk = SkyCoord(ra, dec, unit='deg',  frame='fk5').transform_to('galactic')
        return cls(name, (sk.l.value, sk.b.value))

    @property
    def spectral_model(self):
        if not hasattr(self, 'fit_info'): return None
        modelname = self.fit_info['modelname']
        pars = self.fit_info['pars']
        if modelname=='LogParabola':
            return self.LogParabola(pars)
        elif modelname=='PLSuperExpCutoff':
            return self.PLSuperExpCutoff(pars)
        else:
            raise Exception(f'PointSource: Unrecognized spectral model name {fi["modelname"]}')
    
    def __call__(self, energy):
        """if wtman set, return photon flux at energy"""
        return self.spectral_model(energy) if self.spectral_model else None

    def sed_plot(self, ax=None, figsize=(5,4), **kwargs):
        """Make an SED for the source
        
        - kwargs -- for the Axes object (xlim, ylim, etc.)
        """
        import matplotlib.pyplot as plt
        fig, ax = plt.subplots(figsize=figsize) if ax is None else (ax.figure, ax)
        x =np.logspace(2,5,61)
        y = self(x)
        ax.loglog(x/1e3, y*x**2 * 1e6, '-')
        ax.grid(alpha=0.5)
        kw = dict(xlabel='Energy (GeV)', 
                  ylabel=r'$\mathrm{Energy\ Flux\ (eV\ cm^{-2}\ s^{-1})}$',
                  title=f'{self.name}',
                  xlim=(x.min(),x.max()),
                 )
        kw.update(kwargs)
        ax.set(**kw)
    
    class FluxModel():
        emin, emax = 1e2, 1e5
        def __init__(self, pars, e0=1000):
            self.pars=pars
            self.e0=e0
            
        def photon_flux(self):
            return quad(self, self.emin, self.emax)[0]

        def energy_flux(self):
            func = lambda e: self(e) * e**2
            return quad(func, self.emin, self.emax)[0]

    class LogParabola(FluxModel):

        def __call__(self, e):
            n0,alpha,beta,e_break=self.pars
            x = np.log(e_break/e)
            y = (alpha - beta*x)*x
            return n0*np.exp(y)
        
    class PLSuperExpCutoff(FluxModel):

        def __call__(self,e):
            print('WARNING: check e0!')
            n0,gamma,cutoff,b=self.pars
            return n0*(self.e0/e)**gamma*np.exp(-(e/cutoff)**b)


In [ ]:
show_doc(PointSource)
show_doc(PointSource.fk5)

# TODO: upeate tests if no weight file
# for s, expect in [( PointSource('Geminga'),             'Source "Geminga" at: (l,b)=(195.134,4.266)'),
#                   ( PointSource('gal_source', (0,0)),   'Source "gal_source" at: (l,b)=(0.000,0.000)', ),
#                   ( PointSource.fk5('fk5_source',(0,0)),'Source "fk5_source" at: (l,b)=(96.337,-60.189)',)
#                    ]:    
#     assert str(s)==expect, f'expected {expect}, got {str(s)}'
# PointSource('3C 273').filename, PointSource('3C 273', nickname='3Cxxx').filename

<h2 id="PointSource" class="doc_header"><code>class</code> <code>PointSource</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>PointSource</code>(**`name`**, **`position`**=*`None`*, **`nickname`**=*`None`*, **`config`**=*`None`*)

Manage the position and name of a point source
    

<h4 id="PointSource.fk5" class="doc_header"><code>PointSource.fk5</code><a href="__main__.py#L66" class="source_link" style="float:right">[source]</a></h4>

> <code>PointSource.fk5</code>(**`name`**, **`position`**)

position: (ra,dec) tuple 

In [ ]:
show_doc(check_weights)

<h4 id="check_weights" class="doc_header"><code>check_weights</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>check_weights</code>(**`config`**, **`source`**)

Check that weights for the source are available: if so, return the weight file name

- source -- A PointSource object with information on source location

Returns the filepath to the file if successful, otherwise, print a message abount available files

In [ ]:
#hide
# config = Config()
# if config.valid:
#     print('Check not found, and a good one')
#     test_source = PointSource('test', (0,0), config=config)
#     assert test_source.wtman is None
#     geminga = PointSource('Geminga', config=config)
#     assert geminga.wtman is not None


In [ ]:
# hide
from nbdev.export import notebook2script
notebook2script()
!date

Converted 00_config.ipynb.
Converted 01_data_man.ipynb.
Converted 02_effective_area.ipynb.
Converted 03_sources.ipynb.
Converted 04_exposure.ipynb.
Converted 04_load_data.ipynb.
Converted 04_select_data.ipynb.
Converted 04_simulation.ipynb.
Converted 05_source_data.ipynb.
Converted 06_poisson.ipynb.
Converted 07_loglike.ipynb.
Converted 08_cell_data.ipynb.
Converted 09_lightcurve.ipynb.
Converted 14_bayesian.ipynb.
Converted 90_main.ipynb.
Converted 99_tutorial.ipynb.
Converted index.ipynb.
Sat Jun 26 11:59:48 PDT 2021
